In [11]:
'''проведем тестирование полученной модели на птоке входящих котировок до фильтра'''

import pandas as pd
import random as r
import numpy as np
import datetime
from sklearn import preprocessing
from tqdm import tqdm
tqdm.pandas(desc="")

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [12]:
['AGR_SRC', 'AGR_CREATEDBY', 'OWN_REG', 'AGR_KBM', 'TS_MARK_NAME',
       'TS_MODEL_NAME', 'TS_POWER', 'TS_USE_NAME', 'f1_daysbeforestart',
       'f6_car_age_on_start', 'f13_is_prolongation', 'f14_drivers_count',
       'f15_kbm_min', 'f16_drv_age_min', 'f17_drv_age_max',
       'f18_drv_skill_min', 'f19_drv_skill_max', 'f20_oneperson',
       'f21_agr_days', 'cat']

['AGR_SRC',
 'AGR_CREATEDBY',
 'OWN_REG',
 'AGR_KBM',
 'TS_MARK_NAME',
 'TS_MODEL_NAME',
 'TS_POWER',
 'TS_USE_NAME',
 'f1_daysbeforestart',
 'f6_car_age_on_start',
 'f13_is_prolongation',
 'f14_drivers_count',
 'f15_kbm_min',
 'f16_drv_age_min',
 'f17_drv_age_max',
 'f18_drv_skill_min',
 'f19_drv_skill_max',
 'f20_oneperson',
 'f21_agr_days',
 'cat']

In [13]:
dt = pd.read_excel('../data/calc_input.xlsx')
dt.head()

,AGR_ISN,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_ISN,INS_NAME,INS_SEX,INS_YEAR,...,TS_VIN,TS_POWER,TS_REGNO,TS_USE_NAME,TS_DOC,TS_DOC_SER_NO,AGR_PREVISN,PASS_SCORING,PREMIUMSUM,USE_REG
0,-35187492,2021-05-23,2021-05-24,2022-05-23,ABS-WEB: MAFIN,Web-Osagomafin,-57019624,АФАНАСЬЕВА КАРИНА СЕРГЕЕВНА,М,45.18,...,TMBGC25J593176508,86.0,Н124ХХ178,Личная,NaN,,NaN,Y,3396.26,40.0
1,-35464482,2021-06-05,2021-06-06,2022-06-05,ABS-WEB: MAFIN,Web-Osagomafin,-57668776,ПАНТЕЛЕЕВ ОЛЕГ АНАТОЛЬЕВИЧ,М,48.07,...,XW8ZZZ61ZFG016873,104.0,Н089ВС750,Личная,NaN,,NaN,Y,3459.88,46.0
2,-35906269,2021-05-21,2021-05-22,2022-05-21,ABS-WEB: MAFIN,Web-Osagomafin,-58684888,РЫБИН ИГОРЬ НИКОЛАЕВИЧ,М,58.16,...,Z8TND5FS9DM020651,119.0,Т762АР777,Личная,NaN,,NaN,Y,3516.50,45.0
3,-35740817,2021-05-14,2021-05-15,2022-05-14,ABS-WEB: MAFIN,Web-Osagomafin,-58320805,ГУРЬЯНОВ ИЛЬЯ ВИКТОРОВИЧ,М,34.33,...,XW8ZZZ5NZEG103173,169.0,Т635ЕР50,Личная,NaN,,NaN,Y,6519.35,46.0
4,-35981332,2021-05-17,2021-05-18,2022-05-17,ABS-WEB: MAFIN,Web-Osagomafin,-58862753,КАДЫРОВ ДАМИР МАРСОВИЧ,М,37.58,...,XWEPH81ABK0023714,149.0,М409НУ799,Личная,NaN,,NaN,Y,7884.24,45.0


In [14]:
# оставляем только легковые авто от физлиц
dt = dt[(dt['TS_TYPE'] == 'Легковые автомобили') 
   & (dt['OWN_JUR'] == 0) 
   & (dt['INS_JUR'] == 0)].drop(['TS_TYPE', 'OWN_JUR', 'INS_JUR'], axis=1).drop(dt.columns[[0]], axis = 1)

dt.head()

,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_ISN,INS_NAME,INS_SEX,INS_YEAR,INS_REG,...,TS_VIN,TS_POWER,TS_REGNO,TS_USE_NAME,TS_DOC,TS_DOC_SER_NO,AGR_PREVISN,PASS_SCORING,PREMIUMSUM,USE_REG
0,2021-05-23,2021-05-24,2022-05-23,ABS-WEB: MAFIN,Web-Osagomafin,-57019624,АФАНАСЬЕВА КАРИНА СЕРГЕЕВНА,М,45.18,NaN,...,TMBGC25J593176508,86.0,Н124ХХ178,Личная,NaN,,NaN,Y,3396.26,40.0
1,2021-06-05,2021-06-06,2022-06-05,ABS-WEB: MAFIN,Web-Osagomafin,-57668776,ПАНТЕЛЕЕВ ОЛЕГ АНАТОЛЬЕВИЧ,М,48.07,NaN,...,XW8ZZZ61ZFG016873,104.0,Н089ВС750,Личная,NaN,,NaN,Y,3459.88,46.0
2,2021-05-21,2021-05-22,2022-05-21,ABS-WEB: MAFIN,Web-Osagomafin,-58684888,РЫБИН ИГОРЬ НИКОЛАЕВИЧ,М,58.16,NaN,...,Z8TND5FS9DM020651,119.0,Т762АР777,Личная,NaN,,NaN,Y,3516.50,45.0
3,2021-05-14,2021-05-15,2022-05-14,ABS-WEB: MAFIN,Web-Osagomafin,-58320805,ГУРЬЯНОВ ИЛЬЯ ВИКТОРОВИЧ,М,34.33,NaN,...,XW8ZZZ5NZEG103173,169.0,Т635ЕР50,Личная,NaN,,NaN,Y,6519.35,46.0
4,2021-05-17,2021-05-18,2022-05-17,ABS-WEB: MAFIN,Web-Osagomafin,-58862753,КАДЫРОВ ДАМИР МАРСОВИЧ,М,37.58,NaN,...,XWEPH81ABK0023714,149.0,М409НУ799,Личная,NaN,,NaN,Y,7884.24,45.0


In [15]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600156 entries, 0 to 603773
Data columns (total 39 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   AGR_DATESIGN   600156 non-null  datetime64[ns]
 1   AGR_DATEBEG    600156 non-null  datetime64[ns]
 2   AGR_DATEEND    600156 non-null  datetime64[ns]
 3   AGR_SRC        600156 non-null  object        
 4   AGR_CREATEDBY  600156 non-null  object        
 5   INS_ISN        600156 non-null  int64         
 6   INS_NAME       600156 non-null  object        
 7   INS_SEX        600124 non-null  object        
 8   INS_YEAR       600156 non-null  float64       
 9   INS_REG        201348 non-null  float64       
 10  INS_DOCSER     600144 non-null  object        
 11  OWN_ISN        600156 non-null  int64         
 12  OWN_NAME       600156 non-null  object        
 13  OWN_SEX        600124 non-null  object        
 14  OWN_YEAR       600156 non-null  float64       
 15  

In [7]:
# убираем лишние столбцы

# dt.drop(['TS_REGNO',
#          'TS_VIN',
#          'TS_CATEGORY',
#          'DRV_SEX',
#          'INS_ISN',
#          'OWN_ISN',
#          'INS_SEX',
#          'OWN_SEX',
#          'OWN_DOCSER',
#          'TS_DOC',
#          'TS_DOC_SER_NO'
#         ], axis = 1, inplace = True)

# dt.head()

,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_YEAR,INS_REG,INS_DOCSER,OWN_YEAR,OWN_REG,...,DRV_KBM,AGR_KBM,TS_MARK_NAME,TS_MODEL_NAME,TS_YEAR,TS_POWER,TS_USE_NAME,AGR_PREVISN,PASS_SCORING,PREMIUMSUM
0,2021-05-23 00:00:00,2021-05-24 00:00:00,2022-05-23 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,45.18,NaN,1111,45.18,NaN,...,0.55,0.55,Skoda,Fabia,2009,86.0,Личная,NaN,Y,3396.26
1,2021-05-29 00:00:00,2021-05-30 00:00:00,2022-05-29 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,48.05,NaN,1111,48.05,NaN,...,"0.50,0.50",0.50,Volkswagen,Polo,2014,104.0,Личная,NaN,Y,3641.97
2,2021-05-06 00:00:00,2021-05-07 00:00:00,2022-05-06 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,37.16,NaN,1111,37.16,NaN,...,0.70,0.70,Lada,Vesta,2016,106.0,Личная,NaN,Y,4921.04
3,2021-05-21 00:00:00,2021-05-22 00:00:00,2022-05-21 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,58.16,NaN,1111,58.16,NaN,...,0.50,0.50,CITROЁN,C4,2013,119.0,Личная,NaN,Y,3516.50
4,2021-05-14 00:00:00,2021-05-15 00:00:00,2022-05-14 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,34.33,NaN,1111,34.33,NaN,...,"0.50,0.75",0.75,Volkswagen,Tiguan,2013,169.0,Личная,NaN,Y,6519.35


In [17]:
# после каждого шага сохраняем датасет.
# dt.to_csv('../data/calc_step1.csv')
dt = pd.read_csv('../data/calc_step1.csv')#.drop(dt.columns[[0]], axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (11,17,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
# из дат подписания и начала можно получить информацию о том, насколько заранее куплен полис

def daysbetween(d1, d2):
    dt1 = datetime.datetime.strptime(d1, '%Y-%m-%d')
    dt2 = datetime.datetime.strptime(d2, '%Y-%m-%d')
    return (dt2 - dt1).days

dt['f1_daysbeforestart'] = dt.apply(lambda x: daysbetween(x['AGR_DATESIGN'], x['AGR_DATEBEG']), axis=1)

In [20]:
# из года выпуска получим возраст машины на момент страхования

dt['f6_car_age_on_start'] = dt.apply(lambda x: datetime.datetime.strptime(x['AGR_DATEBEG'], '%Y-%m-%d').year - x['TS_YEAR'], axis=1)

In [21]:
# КБМ по договорам с мультидрайвом отсутствует, заполним его медианой КБМ по другим договорам
dt['AGR_KBM'].fillna(dt['AGR_KBM'].median(), inplace=True)
dt.AGR_KBM.unique()

array([0.55, 0.5 , 0.75, 0.95, 0.85, 0.8 , 0.9 , 0.7 , 0.65, 1.  , 0.6 ,
       1.4 , 1.04, 1.55, 2.3 ])

In [22]:
# выведем отдельно признак пролонгируемого договора
dt['f13_is_prolongation'] = dt['AGR_PREVISN'].apply(lambda x: 1 if x>0 else 0)
dt.f13_is_prolongation.unique()

array([0, 1], dtype=int64)

In [23]:
# отдельными колонками выведем общее количество водителей
# dt['DRV_YEAR'].fillna(0, inplace=True)
dt['f14_drivers_count'] = dt['DRV_YEAR'].apply(lambda x: 0 if x == 0 else len(str(x).split(',')))
dt.f14_drivers_count.unique()

array([ 2,  4,  6, 10,  1,  8,  5,  3, 12,  9,  7, 14, 26, 22, 20, 24, 11,
       16], dtype=int64)

In [24]:
dt.groupby('f14_drivers_count').count()

,Unnamed: 0,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_ISN,INS_NAME,INS_SEX,INS_YEAR,...,TS_USE_NAME,TS_DOC,TS_DOC_SER_NO,AGR_PREVISN,PASS_SCORING,PREMIUMSUM,USE_REG,f1_daysbeforestart,f6_car_age_on_start,f13_is_prolongation
f14_drivers_count,,,,,,,,,,,,,,,,,,,,,
1,246652,246652,246652,246652,246652,246652,246652,246652,246642,246652,...,246652,1969,246652,396,246652,246652,246643,246652,246652,246652
2,196911,196911,196911,196911,196911,196911,196911,196911,196896,196911,...,196911,64740,196911,157,196911,196911,196911,196911,196911,196911
3,1920,1920,1920,1920,1920,1920,1920,1920,1920,1920,...,1920,720,1920,2,1920,1920,1920,1920,1920,1920
4,113947,113947,113947,113947,113947,113947,113947,113947,113940,113947,...,113947,43429,113947,152,113947,113947,113947,113947,113947,113947
5,795,795,795,795,795,795,795,795,795,795,...,795,376,795,0,795,795,795,795,795,795
6,31059,31059,31059,31059,31059,31059,31059,31059,31059,31059,...,31059,13518,31059,44,31059,31059,31059,31059,31059,31059
7,217,217,217,217,217,217,217,217,217,217,...,217,68,217,0,217,217,217,217,217,217
8,7620,7620,7620,7620,7620,7620,7620,7620,7620,7620,...,7620,3538,7620,20,7620,7620,7620,7620,7620,7620
9,53,53,53,53,53,53,53,53,53,53,...,53,18,53,0,53,53,53,53,53,53


In [75]:
# после каждого шага сохраняем датасет.
# dt.to_csv('../data/calc_step2.csv')
dt = pd.read_csv('../data/calc_step2.csv')
dt.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (12,18,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_ISN,INS_NAME,INS_SEX,...,TS_DOC,TS_DOC_SER_NO,AGR_PREVISN,PASS_SCORING,PREMIUMSUM,USE_REG,f1_daysbeforestart,f6_car_age_on_start,f13_is_prolongation,f14_drivers_count
0,0,0,2021-05-23,2021-05-24,2022-05-23,ABS-WEB: MAFIN,Web-Osagomafin,-57019624,АФАНАСЬЕВА КАРИНА СЕРГЕЕВНА,М,...,NaN,,NaN,Y,3396.26,40.0,1,12,0,2
1,1,1,2021-06-05,2021-06-06,2022-06-05,ABS-WEB: MAFIN,Web-Osagomafin,-57668776,ПАНТЕЛЕЕВ ОЛЕГ АНАТОЛЬЕВИЧ,М,...,NaN,,NaN,Y,3459.88,46.0,1,7,0,4
2,2,2,2021-05-21,2021-05-22,2022-05-21,ABS-WEB: MAFIN,Web-Osagomafin,-58684888,РЫБИН ИГОРЬ НИКОЛАЕВИЧ,М,...,NaN,,NaN,Y,3516.50,45.0,1,8,0,2
3,3,3,2021-05-14,2021-05-15,2022-05-14,ABS-WEB: MAFIN,Web-Osagomafin,-58320805,ГУРЬЯНОВ ИЛЬЯ ВИКТОРОВИЧ,М,...,NaN,,NaN,Y,6519.35,46.0,1,8,0,4
4,4,4,2021-05-17,2021-05-18,2022-05-17,ABS-WEB: MAFIN,Web-Osagomafin,-58862753,КАДЫРОВ ДАМИР МАРСОВИЧ,М,...,NaN,,NaN,Y,7884.24,45.0,1,3,0,6


In [71]:
def minmaxfunc(x):
#     try:
#     l = str(x).replace(', ', 'x').replace(',', '.').replace('x', ',').split(',')
    l = str(x).split(',')
    lf = [float(i) for i in l]
    return lf
#     except:
#         return None    

In [63]:
minmaxfunc('6,47, 15,47, 6,47')

[6.47, 15.47, 6.47]

In [77]:
dt.DRV_SKILL = dt.DRV_SKILL.apply(lambda x: str(x).replace(', ', 'r').replace(',', '.').replace('r', ','))
dt.DRV_YEAR = dt.DRV_YEAR.apply(lambda x: str(x).replace(', ', 'r').replace(',', '.').replace('r', ','))

In [78]:
dt[['DRV_YEAR', 'DRV_SKILL']]

,DRV_YEAR,DRV_SKILL
0,45.8,19.48
1,"48.07,38.59","20.52,13.52"
2,58.16,27.48
3,"34.33,31.25","16.33,9.46"
4,"35.95,37.58,38.33","6.47,15.47,6.47"
...,...,...
600151,"42.85,40.5","8.39,15.7"
600152,"35.89,68.46,58.23","14.17,29.48,24.94"
600153,45.41,9.3
600154,45.41,9.3


In [79]:
# отдельными колонками выведем минимальный и максимальный возраст, стаж, и КБМ 
# (КБМ только мин, макс и так в договоре есть)
dt['DRV_KBM'].fillna(dt['AGR_KBM'].median(), inplace=True)
dt['f15_kbm_min'] = dt['DRV_KBM'].apply(lambda x: min(minmaxfunc(x)))
dt['f16_drv_age_min'] = dt['DRV_YEAR'].apply(lambda x: min(minmaxfunc(x)))
dt['f17_drv_age_max'] = dt['DRV_YEAR'].apply(lambda x: max(minmaxfunc(x)))

In [80]:
dt['f18_drv_skill_min'] = dt['DRV_SKILL'].apply(lambda x: min(minmaxfunc(x)))
dt['f19_drv_skill_max'] = dt['DRV_SKILL'].apply(lambda x: max(minmaxfunc(x)))

In [81]:
# дополнительно выведем информацию о том, что собственник = страхователю и он же водит

def oneperson(s):
    if s['INS_NAME'] == s['OWN_NAME'] and s['OWN_NAME'] in str(s['DRV_NAME']).split(','):
        return 1
    else:
        return 0

In [82]:
dt['f20_oneperson'] = dt.apply(oneperson, axis=1)

In [83]:
from collections import Counter

Counter(dt['f20_oneperson'])

Counter({0: 436409, 1: 163747})

In [84]:
# забыл вывести длительность договора
dt['f21_agr_days'] = dt.apply(lambda x: daysbetween(x['AGR_DATEBEG'], x['AGR_DATEEND']), axis=1)

In [85]:
# после каждого шага сохраняем датасет. Крайняя фича - f21
dt.to_csv('../data/v4step1.csv')

In [86]:
dt['INS_YEAR'].fillna(dt['INS_YEAR'].median(), inplace=True)
dt['OWN_YEAR'].fillna(dt['OWN_YEAR'].median(), inplace=True)
dt['f18_drv_skill_min'].fillna(dt['f18_drv_skill_min'].median(), inplace=True)
dt['f19_drv_skill_max'].fillna(dt['f19_drv_skill_max'].median(), inplace=True)
dt['f16_drv_age_min'].fillna(dt['f16_drv_age_min'].median(), inplace=True)
dt['f17_drv_age_max'].fillna(dt['f17_drv_age_max'].median(), inplace=True)

In [87]:
print(dt.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'AGR_DATESIGN', 'AGR_DATEBEG',
       'AGR_DATEEND', 'AGR_SRC', 'AGR_CREATEDBY', 'INS_ISN', 'INS_NAME',
       'INS_SEX', 'INS_YEAR', 'INS_REG', 'INS_DOCSER', 'OWN_ISN', 'OWN_NAME',
       'OWN_SEX', 'OWN_YEAR', 'OWN_REG', 'OWN_DOCSER', 'DRV_NAME', 'DRV_SEX',
       'DRV_YEAR', 'DRV_SKILL', 'DRV_DOCSER', 'DRV_KBM', 'AGR_KBM', 'DRV_ISN',
       'TS_CATEGORY', 'TS_MARK_NAME', 'TS_MODEL_NAME', 'TS_YEAR', 'TS_VIN',
       'TS_POWER', 'TS_REGNO', 'TS_USE_NAME', 'TS_DOC', 'TS_DOC_SER_NO',
       'AGR_PREVISN', 'PASS_SCORING', 'PREMIUMSUM', 'USE_REG',
       'f1_daysbeforestart', 'f6_car_age_on_start', 'f13_is_prolongation',
       'f14_drivers_count', 'f15_kbm_min', 'f16_drv_age_min',
       'f17_drv_age_max', 'f18_drv_skill_min', 'f19_drv_skill_max',
       'f20_oneperson', 'f21_agr_days'],
      dtype='object')


In [95]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600156 entries, 0 to 600155
Data columns (total 52 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           600156 non-null  int64  
 1   Unnamed: 0.1         600156 non-null  int64  
 2   AGR_DATESIGN         600156 non-null  object 
 3   AGR_DATEBEG          600156 non-null  object 
 4   AGR_DATEEND          600156 non-null  object 
 5   AGR_SRC              600156 non-null  object 
 6   AGR_CREATEDBY        600156 non-null  object 
 7   INS_ISN              600156 non-null  int64  
 8   INS_NAME             600156 non-null  object 
 9   INS_SEX              600124 non-null  object 
 10  INS_YEAR             600156 non-null  float64
 11  INS_REG              201348 non-null  float64
 12  INS_DOCSER           600144 non-null  object 
 13  OWN_ISN              600156 non-null  int64  
 14  OWN_NAME             600156 non-null  object 
 15  OWN_SEX          

In [103]:
def reg(x):
    if np.isnan(x['OWN_REG']):
        return str(x['USE_REG'])
    else:
        return str(x['OWN_REG'])

In [106]:
dt['OWN_REG'] = dt.apply(reg, axis=1)

In [107]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600156 entries, 0 to 600155
Data columns (total 53 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           600156 non-null  int64  
 1   Unnamed: 0.1         600156 non-null  int64  
 2   AGR_DATESIGN         600156 non-null  object 
 3   AGR_DATEBEG          600156 non-null  object 
 4   AGR_DATEEND          600156 non-null  object 
 5   AGR_SRC              600156 non-null  object 
 6   AGR_CREATEDBY        600156 non-null  object 
 7   INS_ISN              600156 non-null  int64  
 8   INS_NAME             600156 non-null  object 
 9   INS_SEX              600124 non-null  object 
 10  INS_YEAR             600156 non-null  float64
 11  INS_REG              201348 non-null  float64
 12  INS_DOCSER           600144 non-null  object 
 13  OWN_ISN              600156 non-null  int64  
 14  OWN_NAME             600156 non-null  object 
 15  OWN_SEX          

In [108]:
dt = dt[['AGR_SRC', 'AGR_CREATEDBY', 'OWN_REG', 'AGR_KBM', 'TS_MARK_NAME',
       'TS_MODEL_NAME', 'TS_POWER', 'TS_USE_NAME', 'f1_daysbeforestart',
       'f6_car_age_on_start', 'f13_is_prolongation', 'f14_drivers_count',
       'f15_kbm_min', 'f16_drv_age_min', 'f17_drv_age_max',
       'f18_drv_skill_min', 'f19_drv_skill_max', 'f20_oneperson',
       'f21_agr_days', 'PASS_SCORING', 'PREMIUMSUM']]

In [110]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600156 entries, 0 to 600155
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   AGR_SRC              600156 non-null  object 
 1   AGR_CREATEDBY        600156 non-null  object 
 2   OWN_REG              600156 non-null  object 
 3   AGR_KBM              600156 non-null  float64
 4   TS_MARK_NAME         600156 non-null  object 
 5   TS_MODEL_NAME        600156 non-null  object 
 6   TS_POWER             600156 non-null  float64
 7   TS_USE_NAME          600156 non-null  object 
 8   f1_daysbeforestart   600156 non-null  int64  
 9   f6_car_age_on_start  600156 non-null  int64  
 10  f13_is_prolongation  600156 non-null  int64  
 11  f14_drivers_count    600156 non-null  int64  
 12  f15_kbm_min          600156 non-null  float64
 13  f16_drv_age_min      600156 non-null  float64
 14  f17_drv_age_max      600156 non-null  float64
 15  f18_drv_skill_min

In [111]:
# сохраним итоговый вариант
dt.to_csv('../data/calc.csv')
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600156 entries, 0 to 600155
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   AGR_SRC              600156 non-null  object 
 1   AGR_CREATEDBY        600156 non-null  object 
 2   OWN_REG              600156 non-null  object 
 3   AGR_KBM              600156 non-null  float64
 4   TS_MARK_NAME         600156 non-null  object 
 5   TS_MODEL_NAME        600156 non-null  object 
 6   TS_POWER             600156 non-null  float64
 7   TS_USE_NAME          600156 non-null  object 
 8   f1_daysbeforestart   600156 non-null  int64  
 9   f6_car_age_on_start  600156 non-null  int64  
 10  f13_is_prolongation  600156 non-null  int64  
 11  f14_drivers_count    600156 non-null  int64  
 12  f15_kbm_min          600156 non-null  float64
 13  f16_drv_age_min      600156 non-null  float64
 14  f17_drv_age_max      600156 non-null  float64
 15  f18_drv_skill_min

In [112]:
from catboost import CatBoostClassifier
modelcb = CatBoostClassifier()
modelcb.load_model('../models/cb_model4_v2_06214.cbm', format="cbm")

In [215]:
limit = 0.572309
pred = modelcb.predict_proba(dt.drop(['PASS_SCORING', 'PREMIUMSUM'], axis=1))
res = []
for i in pred:
    if i[0] > limit:
        res.append(0)
    else:
        res.append(1)

In [216]:
Counter(res)

Counter({0: 472132, 1: 128024})

In [217]:
dt['pred'] = res

<ipython-input-217-79985f99520b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['pred'] = res


In [218]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600156 entries, 0 to 600155
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   AGR_SRC              600156 non-null  object 
 1   AGR_CREATEDBY        600156 non-null  object 
 2   OWN_REG              600156 non-null  object 
 3   AGR_KBM              600156 non-null  float64
 4   TS_MARK_NAME         600156 non-null  object 
 5   TS_MODEL_NAME        600156 non-null  object 
 6   TS_POWER             600156 non-null  float64
 7   TS_USE_NAME          600156 non-null  object 
 8   f1_daysbeforestart   600156 non-null  int64  
 9   f6_car_age_on_start  600156 non-null  int64  
 10  f13_is_prolongation  600156 non-null  int64  
 11  f14_drivers_count    600156 non-null  int64  
 12  f15_kbm_min          600156 non-null  float64
 13  f16_drv_age_min      600156 non-null  float64
 14  f17_drv_age_max      600156 non-null  float64
 15  f18_drv_skill_min

In [222]:
okato = pd.read_excel('../data/okato.xlsx')
okato['OKATO'] = okato['OKATO'].apply(lambda x: x/1000000000)
okato.head(100)

def strregion(regid):
    try:
        return list(okato[okato['OKATO'] == regid].REGION)[0]
    except:
        return regid
    
dt['real'] = dt['PREMIUMSUM'].apply(lambda x: 1 if x == 0 else 0)

<ipython-input-222-a2cb548c6b06>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['real'] = dt['PREMIUMSUM'].apply(lambda x: 1 if x == 0 else 0)


In [223]:
dpr = dt[dt['real'] != dt['pred']]
dpr['OWN_REG'] = dpr['OWN_REG'].apply(lambda x: float(x)).apply(strregion)
dpr.head()

<ipython-input-223-7e68c6553bbb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dpr['OWN_REG'] = dpr['OWN_REG'].apply(lambda x: float(x)).apply(strregion)


,AGR_SRC,AGR_CREATEDBY,OWN_REG,AGR_KBM,TS_MARK_NAME,TS_MODEL_NAME,TS_POWER,TS_USE_NAME,f1_daysbeforestart,f6_car_age_on_start,...,f16_drv_age_min,f17_drv_age_max,f18_drv_skill_min,f19_drv_skill_max,f20_oneperson,f21_agr_days,PASS_SCORING,PREMIUMSUM,pred,real
6,ABS-WEB: MAFIN,Web-Osagomafin,Свердловская область,0.80,Hyundai,Accent,102.0,Личная,1,13,...,44.01,44.01,6.47,6.47,0,364,Y,4000.69,1,0
8,ABS-WEB: MAFIN,Web-Osagomafin,Рязанская область,0.95,Lada,Vesta,106.0,Личная,1,4,...,40.40,42.38,1.48,22.40,1,364,Y,13316.20,1,0
12,ABS-WEB: MAFIN,Web-Osagomafin,Город Москва столица Российской Федерации горо...,0.90,Daewoo,Nexia,85.0,Личная,1,15,...,39.87,39.87,2.45,2.45,1,364,Y,16155.68,1,0
32,ABS-WEB: MAFIN,Web-Osagomafin,Нижегородская область,0.80,Honda,Civic,140.0,Личная,2,14,...,24.21,24.21,5.48,5.48,1,364,Y,8196.10,1,0
53,ABS-WEB: MAFIN,Web-Osagomafin,Московская область,0.85,Mitsubishi,Lancer,98.0,Личная,1,15,...,22.31,22.31,4.31,4.31,1,364,Y,6809.59,1,0


In [229]:
diff = dpr[dpr['pred'] != dpr['real']]
Counter(diff['pred'])

Counter({1: 42821, 0: 250733})

In [289]:
diff[(diff.pred == 0) & (diff.AGR_KBM < 1)]

,AGR_SRC,AGR_CREATEDBY,OWN_REG,AGR_KBM,TS_MARK_NAME,TS_MODEL_NAME,TS_POWER,TS_USE_NAME,f1_daysbeforestart,f6_car_age_on_start,...,f16_drv_age_min,f17_drv_age_max,f18_drv_skill_min,f19_drv_skill_max,f20_oneperson,f21_agr_days,PASS_SCORING,PREMIUMSUM,pred,real
77,ABS-WEB: PARTNER SOAP,Web-Пампаду,Иркутская область,0.75,Mazda,Titan,120.0,Личная,4,29,...,35.46,40.95,11.35,16.56,0,364,N,0.0,0,1
78,ABS-WEB: REST API,Сравни.Ру,Рязанская область,0.75,ВАЗ,2192 Kalina,106.0,Личная,4,5,...,43.75,45.91,9.09,24.38,0,364,N,0.0,0,1
79,ABS-WEB: PARTNER SOAP,Web-Нэп,Воронежская область,0.75,Renault,Kaptur,143.0,Личная,5,3,...,35.46,40.95,11.35,16.56,0,364,N,0.0,0,1
81,ABS-WEB: PARTNER SOAP,Web-Save24,Самарская область,0.75,Lada,Priora,97.9,Личная,4,9,...,32.03,34.82,5.28,8.92,1,364,N,0.0,0,1
82,ABS-WEB: PARTNER SOAP,Web-Пампаду,Томская область,0.75,ВАЗ,2121,76.0,Личная,4,37,...,35.46,40.95,11.35,16.56,0,364,N,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600140,ABS-WEB: PARTNER SOAP,Web-Нэп,Удмуртская Республика,0.75,Ford,Transit,155.0,Личная,4,9,...,35.46,40.95,11.35,16.56,0,364,N,0.0,0,1
600141,ABS-WEB: PARTNER SOAP,Web-Ингуру,Город Санкт-Петербург город федерального значения,0.75,Hyundai,Solaris,123.0,Личная,22,3,...,35.46,40.95,11.35,16.56,0,364,N,0.0,0,1
600143,ABS-WEB: PARTNER SOAP,Web-Ингуру,Город Санкт-Петербург город федерального значения,0.75,Hyundai,Solaris,123.0,Личная,23,3,...,35.46,40.95,11.35,16.56,0,364,N,0.0,0,1
600146,ABS-WEB: PARTNER SOAP,Web-Ингуру,Нижегородская область,0.75,Peugeot,307,109.0,Личная,4,15,...,35.46,40.95,11.35,16.56,0,364,N,0.0,0,1


In [235]:
dt[:1]

,AGR_SRC,AGR_CREATEDBY,OWN_REG,AGR_KBM,TS_MARK_NAME,TS_MODEL_NAME,TS_POWER,TS_USE_NAME,f1_daysbeforestart,f6_car_age_on_start,...,f16_drv_age_min,f17_drv_age_max,f18_drv_skill_min,f19_drv_skill_max,f20_oneperson,f21_agr_days,PASS_SCORING,PREMIUMSUM,pred,real
0,ABS-WEB: MAFIN,Web-Osagomafin,40.0,0.55,Skoda,Fabia,86.0,Личная,1,12,...,45.8,45.8,19.48,19.48,0,364,Y,3396.26,0,0


In [285]:
r = dt.drop(['PASS_SCORING', 'PREMIUMSUM'], axis=1).iloc[20]
# r['AGR_KBM'] = 1.5
# r['TS_POWER'] = 220
# r['TS_MARK_NAME'] = 'Toyota'
# r['TS_MODEL_NAME'] = 'Camry'
# r['f6_car_age_on_start'] = 1
# r['f1_daysbeforestart'] = 10
# r['OWN_REG'] = '05'
print(r)
print('')

# r = dt.drop(['PASS_SCORING', 'PREMIUMSUM'], axis=1).iloc[10].values
tst = modelcb.predict_proba(r)
if tst[0] > limit:
    print('Продаём')
else:
    print('Не продаём')

AGR_SRC                ABS-WEB: MAFIN
AGR_CREATEDBY          Web-Osagomafin
OWN_REG                             5
AGR_KBM                           0.5
TS_MARK_NAME                  Peugeot
TS_MODEL_NAME                     206
TS_POWER                           74
TS_USE_NAME                    Личная
f1_daysbeforestart                  1
f6_car_age_on_start                14
f13_is_prolongation                 0
f14_drivers_count                   6
f15_kbm_min                       0.5
f16_drv_age_min                 43.06
f17_drv_age_max                 67.93
f18_drv_skill_min               21.47
f19_drv_skill_max               47.47
f20_oneperson                       1
f21_agr_days                      364
pred                                0
real                                0
Name: 20, dtype: object

Продаём
